In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as stats

from nonlinear import initial_distribution, transition_model, observation_model
from filters import ensemble_kf, bootstrap_pf, log_likelihood, sample_trajectory

In [ ]:
def particle_metropolis_hastings(y, n_iters, n_particles, p_theta, q, init_dist, tr_model, obs_model):
    uniform_rvs = stats.uniform.rvs(size=n_iters)

    theta0 = p_theta.rvs()
    theta = np.zeros((n_iters + 1, len(theta0)))
    theta[0] = theta0

    log_ls = np.zeros((n_iters + 1,))

    for k in range(n_iters + 1):
        if k == 0:
            theta_candidate = theta0
        else:
            theta_candidate = q(theta[k - 1]).rvs()

        p0 = init_dist(theta_candidate)
        p = tr_model(theta_candidate)
        nu = obs_model(theta_candidate)

        u, _, _ = bootstrap_pf(y, n_particles, p0, p, nu)

        prob_y = nu(u).pdf(y[:, np.newaxis]).mean(axis=1)
        log_l = np.sum(np.log(prob_y))

        if k == 0:
            log_ls[0] = log_l
            continue

        alpha = (
                 p_theta.pdf(theta_candidate) / p_theta.pdf(theta[k - 1]) *
                 np.exp(log_l - log_ls[k - 1]) *
                 q(theta_candidate).pdf(theta[k - 1]) / q(theta[k - 1]).pdf(theta_candidate)
        ).prod()
        alpha = min(1, alpha)

        if uniform_rvs[k - 1] < alpha:
            theta[k] = theta_candidate
            log_ls[k] = log_l
        else:
            theta[k] = theta[k - 1]
            log_ls[k] = log_ls[k - 1]
        print(theta[k])
    return theta

In [ ]:
n_iters = 100
n_particles = 2 ** 6
p_theta = stats.uniform([0.1, 0.6], [0.2, 0.2]) # stats.invgamma([3, 3], scale=[0.5, 0.5])
q = lambda th: stats.truncnorm(-th/0.1, np.inf, loc=th, scale=0.1)
init_dist = lambda th: initial_distribution(phi, th[0])
tr_model = lambda th: transition_model(phi, th[0])
obs_model = lambda th: observation_model(th[1])

theta = particle_metropolis_hastings(y, n_iters, n_particles, p_theta, q, init_dist, tr_model, obs_model)

In [ ]:
plt.scatter(np.arange(n_iters + 1), theta[:, 0], linewidth=1)
plt.scatter(np.arange(n_iters + 1), theta[:, 1], linewidth=1)
plt.show()